## Merge geom and animal files

## Formatting

In [242]:
## Merges geom and animal files with the same file name

import os
import pandas as pd

# Make sure file names in animal and geom have the exact same name, !!! otherwise it wont join the files !!!
main_folder = r"C:\Users\topohl\Documents\GitHub\sleap\docs\notebooks\test merge"
geom_folder = os.path.join(main_folder, "geom")
animal_folder = os.path.join(main_folder, "animal")

# Get list of all CSV files in the geom folder
geom_files = [f for f in os.listdir(geom_folder) if f.endswith('.csv')]

# Loop through CSV files and merge with matching animal file
for geom_file in geom_files:
    animal_file = geom_file.replace("geom", "animal")
    animal_path = os.path.join(animal_folder, animal_file)
    if os.path.exists(animal_path):
        geom_path = os.path.join(geom_folder, geom_file)
        geom_df = pd.read_csv(geom_path)
        animal_df = pd.read_csv(animal_path)
        merged_df = pd.concat([geom_df, animal_df], axis=1)
        merged_path = os.path.join(main_folder, geom_file.replace("analysis", "merged"))
        merged_df.to_csv(merged_path, index=False)
        # add output text for each file when processed
        print("Merged file: " + merged_path)

In [243]:
## Merges geom and animal files with the same file name
#  also formats table based on headers
#  formats table to have format thats readable by DLCanalyzer
#  Stores the merged and formatted csv sheet into the main folder directory

import pandas as pd
import os

# get the path to the directory containing the CSV files
# the directory contains both the animal and the geom folder, e.g. path\to\directory\animal
directory = r"path\to\directory"
output_folder = r"path\to\directory\formatted"

# Create the 'formatted' folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# loop through all CSV files in the directory
for filename in os.listdir(directory):
    if filename.endswith(".csv") and "merged" in filename:
        file_path = os.path.join(directory, filename)
        file_name = os.path.splitext(filename)[0]  # get file name without extension
        # load the CSV file into a dataframe
        df = pd.read_csv(os.path.join(directory, filename))
        
        # create the MultiIndex for the columns
        columns = pd.MultiIndex.from_tuples([(c.split("_")[0], c.split("_")[1]) for c in df.columns])

        # create the new dataframe with the split header
        new_df = pd.DataFrame(columns=columns)

        # iterate over each original column and split the header
        for col in df.columns:
            prefix, suffix = col.split("_")
            new_df[prefix, suffix] = [prefix, suffix] + list(df[col].values)

        # set the first row as the column headers
        new_df.columns = new_df.iloc[0]

        # insert an empty column in front of the first existing column
        new_df.insert(loc=0, column="", value="")

        # insert a new column at the beginning with "bodyparts" and "coords"
        new_df.iloc[0, 0] = "bodyparts"
        new_df.iloc[1, 0] = "coords"

        # shift "bodyparts" and "coords" up by one cell
        new_df.iloc[1, 0] = new_df.iloc[1, 0]
        new_df.iloc[2, 0] = ""

        # insert an empty row at the top of the dataframe
        empty_row = pd.DataFrame([[""] * len(new_df.columns)], columns=new_df.columns)
        new_df = pd.concat([empty_row, new_df], ignore_index=True)

        # reset the column labels
        new_df.columns = new_df.iloc[0]
        new_df = new_df.drop(new_df.index[0])

        # reset headers to standard format
        new_header_names = ["column_" + str(i) for i in range(1, len(new_df.columns)+1)]
        new_df.columns = new_header_names

        # assign range of values to the first column below "coords"
        new_df.iloc[2:, 0] = range(len(new_df)-2)

        # insert new columns and set their values
        for i in range(2, len(new_df.columns)+26):
            if (i-2) % 3 == 0:
                new_col_name = new_df.columns[i] + "_new" 
                insert_pos = new_df.columns.get_loc(new_df.columns[i]) + 1
                new_df.insert(i+1, new_col_name, "")
                new_df.iloc[0, i+1] = new_df.iloc[0, i]
                new_df.iloc[1, i+1] = "likelihood"
                new_df.iloc[2:, i+1] = 1
        
        # save the formatted dataframe as a new CSV file
        new_file_name = file_name.split("_", 6)[-1] + "_formatted.csv"
        new_df.to_csv(os.path.join(output_folder, new_file_name), index=False)

        # add output text for each file when processed
        print("Formatted file: " + new_file_name)